### Import and config

In [2]:
# Imports
import os
import logging
from datetime import timezone, timedelta

from dotenv import load_dotenv

import pandas as pd
import numpy as np

from sqlalchemy import create_engine, text, Integer, Float, String, Boolean, DateTime, Interval, Text, BigInteger


# Configuration
load_dotenv()

# DB
DB_URI = os.getenv('DB_URI')

# Silver tables
TARGET_S_SCHEMA = os.getenv('TARGET_S_SCHEMA')
ACTIVITIES_S_TABLE = os.getenv('ACTIVITIES_S_TABLE')

# Gold tables
TARGET_G_SCHEMA = os.getenv('TARGET_S_SCHEMA')
DIM_CALENDAR_TABLE = os.getenv('DIM_CALENDAR_TABLE')

# Other
LOG_LEVEL = os.getenv('LOG_LEVEL')

logging.basicConfig(
    level=getattr(logging, LOG_LEVEL.upper(), logging.INFO),
    format="%(asctime)s | %(levelname)s | %(message)s"
)

pd.set_option('display.max_columns', None)

### DB names validation

In [3]:
REQUIRED_DB_ENV = ['DB_URI', 'TARGET_S_SCHEMA','ACTIVITIES_S_TABLE', 'TARGET_S_SCHEMA', 'DIM_CALENDAR_TABLE']
missing_db_env = [env for env in REQUIRED_DB_ENV if not os.getenv(env)]
if missing_db_env:
  raise RuntimeError(f"Missing env variables: {', '.join(missing_db_env)}.")

### Request data from `silver` layer

In [4]:
engine = create_engine(
  DB_URI, 
  pool_pre_ping=True, 
  pool_size=5, 
  max_overflow=10
)
logging.info("Connection established")

2025-09-19 09:24:38,719 | INFO | Connection established


In [5]:
with engine.begin() as conn:
  activities_df = pd.read_sql(text(f"SELECT * FROM {TARGET_S_SCHEMA}.{ACTIVITIES_S_TABLE}"), conn)
logging.info(f"Data from {TARGET_S_SCHEMA}.{ACTIVITIES_S_TABLE} downloaded.")

2025-09-19 09:24:40,098 | INFO | Data from silver.activities downloaded.


In [6]:
activities_df.head()

,id,name,start_date_dt,start_date_local_dt,distance,moving_time,moving_time_td,elapsed_time,elapsed_time_td,total_elevation_gain,elev_low,elev_high,type,sport_type,workout_type,achievement_count,kudos_count,comment_count,athlete_count,photo_count,trainer,commute,manual,visibility,average_speed,avg_pace_str,avg_pace_float,max_speed,max_pace_str,max_pace_float,average_cadence,average_watts,max_watts,weighted_average_watts,has_heartrate,average_heartrate,max_heartrate,pr_count,total_photo_count,suffer_score,description,calories,device_name,start_lat,start_lng,map_id,gear_id,location_id
0,15831049874,Afternoon Weight Training,2025-09-16 13:01:07+00:00,2025-09-16 13:01:07+00:00,0.0,3825,0 days 01:03:45,3825,0 days 01:03:45,0.0,0.0,0.0,Workout,WeightTraining,NaN,0,5,1,1,0,True,False,False,followers_only,0.000,None,NaN,0.00,None,NaN,NaN,NaN,NaN,NaN,True,94.0,222.0,0,0,9.0,Reska8️⃣8️⃣\nBench press PR: 85kg🥳,254.0,Garmin Forerunner 970,NaN,NaN,a15831049874,None,NaN
1,15820198827,Tempo 2km🥵,2025-09-15 14:23:21+00:00,2025-09-15 14:23:21+00:00,9521.9,3241,0 days 00:54:01,3241,0 days 00:54:01,13.0,115.4,125.0,Run,Run,3.0,0,8,0,1,0,False,False,False,everyone,2.938,5:40,5.672793,4.94,3:22,3.373819,167.8,353.5,493.0,369.0,True,153.1,178.0,0,0,66.0,Tempo 2km Repeats with Runna ✅\n\nWyszło troch...,735.0,Garmin Forerunner 970,51.107301,17.124098,a15820198827,g24134620,1179.0
2,15805849875,15km Long Run☔️,2025-09-14 07:59:25+00:00,2025-09-14 07:59:25+00:00,15059.0,5461,0 days 01:31:01,5488,0 days 01:31:28,31.0,114.6,125.4,Run,Run,2.0,0,4,0,1,0,False,False,False,everyone,2.758,6:03,6.043026,3.44,4:51,4.844961,172.0,335.5,455.0,334.0,True,144.8,153.0,0,0,66.0,15km Long Run with Runna ✅\n\nOkrutny beton po...,1170.0,Garmin Forerunner 970,51.107336,17.124136,a15805849875,g24134620,1179.0
3,15798063578,Tempo 4km😮‍💨,2025-09-13 13:59:16+00:00,2025-09-13 13:59:16+00:00,7531.2,2400,0 days 00:40:00,2400,0 days 00:40:00,11.0,115.2,125.2,Run,Run,3.0,0,7,0,1,0,False,False,False,everyone,3.138,5:19,5.311239,4.98,3:21,3.346720,167.8,364.2,509.0,377.0,True,155.3,181.0,0,0,53.0,Tempo 4km with Runna ✅\n\nWygrana walka z wiat...,579.0,Garmin Forerunner 970,51.107367,17.124221,a15798063578,g24134620,1179.0
4,15786538213,6km Easy Run😌,2025-09-12 13:40:46+00:00,2025-09-12 13:40:46+00:00,6062.9,2140,0 days 00:35:40,2140,0 days 00:35:40,17.0,111.2,123.0,Run,Run,NaN,0,7,0,1,0,False,False,False,everyone,2.833,5:53,5.883045,3.56,4:41,4.681648,166.2,351.5,445.0,350.0,True,143.7,152.0,0,0,24.0,6km Easy Run with Runna ✅\n\nOdgruzowywanie nó...,469.0,Garmin Forerunner 970,51.107330,17.124162,a15786538213,g24134620,1179.0


### Setup `gold.dim_calendar`